# Nova Hyperpod RIG — One-Time Cluster & Dependency Setup
This notebook handles the one-time (or infrequent) setup required to run Nova workloads on SageMaker Hyperpod with RIG. It walks through cluster configuration and validation, IAM/permissions checks, and environment + dependency installation so your workspace is ready for repeatable training and evaluation.


## Prerequisites

- To get started, you need to create SageMaker Studio JupyterLab space
    - Go to AWS console
    - Go to Sagemaker AI -> Domains -> Set up for single user (Quick setup) -> Submit
    - This will open SageMaker studio
    - Go to JupyterLab -> Create JupyterLab space -> Submit
    - Once the space is created -> Open JupyterLab and upload this notebook

## Cell Execution Guide

- 🔵 **ONE-TIME SETUP**: Run once per notebook JupyterLab instance
- 🟣 **KERNEL RESTART**: Run everytime kernel restarts
- 🟡 **PER-JOB**: Run for every training/evaluation job

## 🔵 ONE-TIME SETUP: Cluster setup

- Setup time: 5 mins
- Creation time: 45 mins

SMHP RFT uses Hyperpod RIG cluster which requires setting up the cluster.

#### Step 1:
Before setting up the cluster, you will need procure GPU instances (p5.48xlarge, p4d.24xlarge, g5.12xlarge etc.) based on the training technique. Plese contact AWS support to procure the instances

#### Step 2:
Follow steps in this doc with following changes: https://catalog.us-east-1.prod.workshops.aws/workshops/dcac6f7a-3c61-4978-8344-7535526bf743/en-US/02-smhp-rig/00-setup/00-new-console-experience
- In `Add instance group` select GPU instance you want
    - Select only 1 instance count for now. We can scale once cluster is created
- `Throughput per unit of storage` as 500
- `Storage capacity` based on the number of experiments you will do. For starters, it can be 9.6
- Click Submit
- Wait for the cluster to be created. It can take 10 - 45 mins.
- Once the cluster is created, go to Amazon SageMaker AI -> Model training & customization -> HyperPod clusters -> click on the cluster you created -> Edit -> Click on the instance group -> Edit -> Change `Quantity` to desired instance counts -> Submit

#### Common Issues

##### Cluster creation
If the cluster creation fails, go to `CloudFormation` and check the stack for failing reason. If it failed for something like, `We currently do not have sufficient capacity to launch new ml.p5.48xlarge instances. Please try again`, it means you don't have capacity (ODCR) for the instance type e.g. `ml.p5.48xlarge`. Please follow step 1

## 🔵 ONE-TIME SETUP: Permissions

#### Setting up all the env variables
- RIG_PRETTY_CLUSTER_NAME: AWS console -> Amazon SageMaker AI -> HyperPod clusters -> your cluster
- EKS_CLUSTER_ROLE_ARN: AWS console -> Amazon SageMaker AI -> HyperPod clusters -> your cluster -> Details -> EKS cluster IAM role ARN
- EKS_CLUSTER_NAME: AWS console -> Amazon SageMaker AI -> HyperPod clusters -> your cluster -> Orchestrator
- EKS_CLUSTER_ARN: AWS console -> Amazon SageMaker AI -> HyperPod clusters -> your cluster -> view EKS -> Get the ARN
- SAGEMAKER_STUDIO_ROLE -> Amazon SageMaker AI -> Domains -> your domain -> Authentication and permissions -> Default execution role

In [ ]:
%env RIG_PRETTY_CLUSTER_NAME=<RIG_PRETTY_CLUSTER_NAME>
%env EKS_CLUSTER_ROLE_ARN=<EKS_CLUSTER_ROLE_ARN>
%env EKS_CLUSTER_NAME=<EKS_CLUSTER_NAME>
%env EKS_CLUSTER_ARN=<EKS_CLUSTER_ARN>

### Permissions for Sagemaker Studio role

- Go to IAM -> roles -> SAGEMAKER_STUDIO_ROLE -> Add permissions -> Attach Policies -> Attack the following:
  - AmazonEKSClusterPolicy
  - AmazonEKSMCPReadOnlyAccess
  - CloudWatchFullAccessV2

#### Access Entry manager

EKS requires an explicit access entry + access policy to authorize an IAM principal at the Kubernetes layer.
Without this mapping, kubeconfig and token generation may succeed, but the Kubernetes API will still return 401 Unauthorized.

Steps:
- Add the following policy to SAGEMAKER_STUDIO_ROLE (Update the ACCOUNT and EKS_CLUSTER_NAME with the actual name before adding)

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ManageEKSAccessEntriesForHyperPodCluster",
            "Effect": "Allow",
            "Action": [
                "eks:CreateAccessEntry",
                "eks:UpdateAccessEntry",
                "eks:DeleteAccessEntry",
                "eks:DescribeAccessEntry",
                "eks:ListAccessEntries",
                "eks:AssociateAccessPolicy",
                "eks:DisassociateAccessPolicy",
                "eks:ListAssociatedAccessPolicies"
            ],
            "Resource": "arn:aws:eks:us-east-1:<ACCOUNT>:cluster/<EKS_CLUSTER_NAME>"
        },
        {
            "Sid": "AssociatePoliciesToAccessEntriesForThisCluster",
            "Effect": "Allow",
            "Action": [
                "eks:AssociateAccessPolicy",
                "eks:DisassociateAccessPolicy",
                "eks:ListAssociatedAccessPolicies"
            ],
            "Resource": "arn:aws:eks:us-east-1:<ACCOUNT>:access-entry/<EKS_CLUSTER_NAME>/*"
        }
    ]
}

In [ ]:
%%bash

aws eks create-access-entry \
  --cluster-name $EKS_CLUSTER_NAME \
  --principal-arn $SAGEMAKER_STUDIO_ROLE

In [ ]:
%%bash

aws eks associate-access-policy \
  --cluster-name $EKS_CLUSTER_NAME \
  --principal-arn $SAGEMAKER_STUDIO_ROLE \
  --policy-arn arn:aws:eks::aws:cluster-access-policy/AmazonEKSClusterAdminPolicy \
  --access-scope type=cluster

### EKS cluster permissions

Add the follwoing permissions to the EKS_CLUSTER_ROLE_ARN

- Go to IAM -> roles -> EKS_CLUSTER_ROLE_ARN -> Add permissions -> Attach Policies -> Attack the following:
  - AmazonSageMakerClusterInstanceRolePolicy
  - AWSLambdaRole

#### Dataset access
Grant cluster access to your s3 bucket for job input and output

- Add the following policy to EKS_CLUSTER_ROLE_ARN (Update the S3_BUCKET with the actual name before adding)

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ListBucket",
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket",
                "s3:GetBucketLocation"
            ],
            "Resource": "arn:aws:s3:::<S3_BUCKET>"
        },
        {
            "Sid": "ReadWriteObjects",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:GetObjectVersion",
                "s3:PutObject"
            ],
            "Resource": "arn:aws:s3:::<S3_BUCKET>"
        }
    ]
}

## 🔵 ONE-TIME SETUP: Installing Dependencies

### Setup virtual environment

Follow these steps to create a Python virtual environment and register it as a Jupyter kernel in a SageMaker Amazon Linux 2 notebook.

1️⃣ Open a Terminal

From JupyterLab:

Click on the + icon on the top left → Terminal

2️⃣ Create a virtual environment

Run the following in the Terminal:

```
python3 -m venv nova_sdk
```

3️⃣ Activate the virtual environment
```
source nova_sdk/bin/activate
```


You should see the environment name in your prompt:

`(nova_sdk) [ec2-user@ip-xxx SageMaker]$`

4️⃣ Install ipykernel inside the environment
`pip install --upgrade pip ipykernel`

5️⃣ Register the environment as a Jupyter kernel

Still inside the activated environment:

```
python -m ipykernel install --user \
  --name nova_sdk \
  --display-name "Python (nova_sdk)"
```


Expected output:

`Installed kernelspec nova_sdk`

6️⃣ Refresh Jupyter and select the kernel

- Refresh the browser page

- Open your notebook

- Go to Kernel → Change Kernel

- Select Python (nova_sdk)


In [1]:
# Verify the new env

import sys
sys.executable

## You should see `/home/ec2-user/SageMaker/nova_sdk/bin/python`

'/home/sagemaker-user/nova_sdk/bin/python'

In [2]:
%%bash

# Update PATH in bash with new kernel path

echo 'export PATH="$HOME/nova_sdk/bin:$PATH"' >> ~/.bashrc
echo 'export PATH="$HOME/nova_sdk/bin:$PATH"' >> ~/.bash_profile
source ~/.bashrc

### Update paths (IMPORTANT)

In [3]:
# Update paths so that Python and CLI commands use the depenedncies installed in nova_sdk venv

import os
os.environ["BASH_ENV"] = os.path.expanduser("~/.bashrc")
os.environ["PATH"] = f"{os.path.expanduser('~')}/.local/bin:" + os.environ["PATH"]

venv_bin = f"{os.environ["HOME"]}/nova_sdk/bin"
os.environ["PATH"] = f"{venv_bin}:" + os.environ["PATH"]

# Set paths updted flag
paths_updated = True

In [4]:
# Verify python points to /home/sagemaker-user/nova_sdk/bin/python
!which python
!which pip

/home/sagemaker-user/nova_sdk/bin/python
/home/sagemaker-user/nova_sdk/bin/pip


### Install Hyperpod CLI

In [5]:
## Clone the Hyperpod CLI repo
!git clone --recursive https://github.com/aws/sagemaker-hyperpod-cli.git

fatal: destination path 'sagemaker-hyperpod-cli' already exists and is not an empty directory.


In [6]:
## Install HP CLI (release_v2 branch)

!cd sagemaker-hyperpod-cli && git checkout release_v2 && /home/sagemaker-user/nova_sdk/bin/pip install .

M	src/hyperpod_cli/sagemaker_hyperpod_recipes
Already on 'release_v2'
Your branch is up to date with 'origin/release_v2'.
Processing /home/sagemaker-user/sagemaker-hyperpod-cli
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hyperpod: filename=hyperpod-2.0.0-py3-none-any.whl size=7017867 sha256=123634d5640d19b59e89ac47f34f524013c888750c36e690ca951b265060a20f
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/f9/9e/af/1b12c7244658bcdbefcc7bfea6141c3c1ba67e9d8efe1337de
Successfully built hyperpod
  Attempting uninstall: hyperpod
    Found existing installation: hyperpod 2.0.0
    Uninstalling hyperpod-2.0.0:
      Successfully uninstalled hyperpod-2.0.0

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
# Verify hyperpod installation. Should be in /home/sagemaker-user/nova_sdk/bin
!which hyperpod

/home/sagemaker-user/nova_sdk/bin/hyperpod


### Install Nova SDK

The Nova Customization SDK is a comprehensive Python SDK for customizing Amazon Nova models. The SDK provides a unified interface for training, evaluation, monitoring, deployment, and inference of Amazon Nova models across different platforms including SageMaker AI and Amazon Bedrock. Whether you're adapting models to domain-specific tasks or optimizing performance for your use case, this SDK provides everything you need in one unified interface.

In [ ]:
!pip install amzn-nova-customization-sdk

In [9]:
# Should be in /home/sagemaker-user/nova_sdk/lib

!pip show amzn-nova-customization-sdk

Name: amzn-nova-customization-sdk
Version: 1.0.72
Summary: A Python SDK for customizing Amazon Nova models.
Home-page: 
Author: 
Author-email: Amazon <nova-customizersdk@amazon.com>
License-Expression: Apache-2.0
Location: /home/sagemaker-user/nova_sdk/lib/python3.12/site-packages
Requires: boto3, sagemaker
Required-by: 


### Install Kubectl

In [10]:
%%bash
set -euo pipefail

curl -LO https://dl.k8s.io/release/$(curl -Ls https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl

chmod +x kubectl
mkdir -p ~/.local/bin
mv kubectl ~/.local/bin/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.8M  100 55.8M    0     0   137M      0 --:--:-- --:--:-- --:--:--  137M


In [11]:
# verify kubectl. Should be in /home/sagemaker-user/.local/bin
!which kubectl

/home/sagemaker-user/.local/bin/kubectl


### Install AWS CLI V2

In [12]:
%%bash
set -euo pipefail

# Remove v1
sudo rm -rf /opt/conda/bin/aws
rm -rf /home/sagemaker-user/nova_sdk/bin/aws

curl -fsSL "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o /tmp/awscliv2.zip
unzip -q /tmp/awscliv2.zip -d /tmp
sudo /tmp/aws/install --bin-dir /usr/local/bin --install-dir /usr/local/aws-cli --update

rm -rf /tmp/awscliv2.zip /tmp/aws

You can now run: /usr/local/bin/aws --version


In [13]:
# Verify CLI version is v2

!which aws
!aws --version

/usr/local/bin/aws
aws-cli/2.33.21 Python/3.13.11 Linux/6.1.159-181.297.amzn2023.x86_64 exe/x86_64.ubuntu.22


### Installing Helm

In [14]:
%%bash

curl -fsSL https://get.helm.sh/helm-v3.14.2-linux-amd64.tar.gz -o helm.tar.gz
tar -zxvf helm.tar.gz
mv linux-amd64/helm ~/.local/bin/helm
chmod +x ~/.local/bin/helm

linux-amd64/
linux-amd64/README.md
linux-amd64/helm
linux-amd64/LICENSE


In [15]:
# Verify helm. Should be in /home/sagemaker-user
!which helm

/home/sagemaker-user/.local/bin/helm


### [Optional] MLFlow Setup 

To use MLFlow with your traininer jobs, you need to create a MLFlow App:
    
    - Go to AWS console
    - Go to Sagemaker AI -> Sagemaker Studio -> Click on Domain created in step 1 -> Open Studio
    - Go to Applications -> MlFlow -> Create MlFlow App -> Select name and create

### Delete problematic resources

Cluster creation through UI currently has a bug where it deploys `kueue` pods which are not supported by Hyperpod RIG. We need to delete them

In [ ]:
%%bash

## Get the name of the pod

kubectl delete deployment kueue-controller-manager -n kueue-system 
kubectl delete mutatingwebhookconfiguration kueue-mutating-webhook-configuration
kubectl delete validatingwebhookconfiguration kueue-validating-webhook-configuration